In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

In [ ]:
import numpy,random
from scipy.stats import norm
from scipy.stats import multivariate_normal

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})


# null hypothesis is a single source - multi normal Gaussian 
mu_H0_x=mu_H0_y=mu_H0_z = 0
sigma_H0_x=sigma_H0_y=sigma_H0_z = 1

# alt hypothesis is a different single source - multinormal Gaussian 
mu_H1_x = 0.5
mu_H1_y = -0.3
mu_H1_z = 0.2

sigma_H1_x = 0.4 
sigma_H1_y = 0.3
sigma_H1_z = 0.7

# correlations 
rXY = 0.3
rXZ = 0.6
rYZ = 0.5 


In [ ]:
n_events = 100000

def makeCovariance(sx,sy,sz):
    cov = [  [sx**2    , rXY*sy*sz, rXZ*sx*sz]
            ,[rXY*sy*sz, sy**2,     rYZ*sz*sy]
            ,[rXZ*sx*sz, rYZ*sz*sy, sz**2    ]
          ]
    print(cov)
    return cov
           
pH0 = multivariate_normal([mu_H0_x,mu_H0_y,mu_H0_z], makeCovariance(sigma_H0_x,sigma_H0_y,sigma_H0_z))
pH1 = multivariate_normal([mu_H1_x,mu_H1_y,mu_H1_z], makeCovariance(sigma_H1_x,sigma_H1_y,sigma_H1_z), allow_singular=True)

In [ ]:
net_main = torch.nn.Sequential(
        torch.nn.Linear(3, 20),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(20, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 20),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(20, 1),
        torch.nn.Sigmoid()
    )




net_wS = torch.nn.Sequential(
    torch.nn.Linear(1,1)
)

# Re-weight events based on f(x)=net

def weight(x):
    #x = torch.reshape([x],(1,1))
    #x = torch.reshape(torch.from_numpy(numpy.array([x])).type(torch.double),(1,1))
    f = net_main(x)
    w = (f/(1-f))
    return w.detach()



net_main.to(torch.double)
#net_other.to(torch.double)

optimizer_main  = torch.optim.Adam(net_main.parameters())
#optimizer_other = torch.optim.Adam(net_other.parameters())

#loss_func = torch.nn.BCELoss() 

#import ks2weights
#def custom_Loss(x,y):
    # calculate the weights at the current network
    # x should be MC part (H0 weighted), y should be original(H0 unweighted), but I only want to look at the 3rd variable here. 
    
#    xx = x[:,2].numpy()
#    return ks2weights.D2(xx,yy,w)
#    yy = y[:,2].numpy()
    

In [ ]:
# make some data, should be from H0 and H1 
xH0r = pH0.rvs(size=n_events)
xH1r = pH1.rvs(size=n_events)
fH0 = [0 for xx in xH0r]
fH1 = [1 for xx in xH1r]

xH0 = torch.reshape(torch.from_numpy(numpy.array(xH0r)).type(torch.double),(n_events,3))

xH1 = torch.reshape(torch.from_numpy(numpy.array(xH1r)).type(torch.double),(n_events,3))

x = torch.from_numpy(numpy.concatenate([xH0,xH1])).type(torch.double)
f = torch.from_numpy(numpy.concatenate([fH0,fH1])).type(torch.double)


x = torch.reshape(x,(n_events*2,3))

f = torch.reshape(f,(n_events*2,1))

x_z = x[:,2:3]

print(xH0[:,2])

In [ ]:
# make some 2D hists
fig, axs = plt.subplots(2,3,figsize=(18,12))

#axs[0][0].hist2d(xH0[:,0],xH0[:,1],bins=100)
axs[0][0].set_xlabel("$X$")
axs[0][0].set_ylabel("$Y$")

#axs[0][1].hist2d(xH0[:,0],xH0[:,2],bins=100)
axs[0][1].set_xlabel("$X$")
axs[0][1].set_ylabel("$Z$")

#axs[0][2].hist2d(xH0[:,1],xH0[:,2],bins=100)
axs[0][2].set_xlabel("$Y$")
axs[0][2].set_ylabel("$Z$")


#axs[1][0].hist2d(xH1[:,0],xH1[:,1],bins=100)
axs[1][0].set_xlabel("$X$")
axs[1][0].set_ylabel("$Y$")

#axs[1][1].hist2d(xH1[:,0],xH1[:,2],bins=100)
axs[1][1].set_xlabel("$X$")
axs[1][1].set_ylabel("$Z$")

#axs[1][2].hist2d(xH1[:,1],xH1[:,2],bins=100)
axs[1][2].set_xlabel("$Y$")
axs[1][2].set_ylabel("$Z$")

plt.savefig("model.pdf")

In [ ]:
# make a histogram of the z to make weights 

bins_z = numpy.linspace(-4,4,50)
values_H0 = plt.hist(xH0[:,2].numpy(),bins=bins_z,histtype='step',label="$H_{0}$",color='red')
values_H1 = plt.hist(xH1[:,2].numpy(),bins=bins_z,histtype='step', label="$H_{1}$",color='blue')

print(values_H0)
print(values_H1)


def slow_find_weight_z(z):
    if z < -4 :return 0
    if z >= 4 :return 0
    id = numpy.digitize([z], bins_z)[0]-1
    #print(z,id)
    if values_H0[0][id] == 0 : return 1 
    return values_H1[0][id]/values_H0[0][id]

find_weight_z = numpy.vectorize(slow_find_weight_z)

print(find_weight_z(xH0[:,2].numpy()))

plt.clf()

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,6))
ax1.hist(xH0[:,2].numpy(),bins=bins_z,histtype='step',label="$H_{0}$",color='red')
ax1.hist(xH1[:,2].numpy(),bins=bins_z,histtype='step', label="$H_{1}$",color='blue')
ax1.set_xlabel("$Z$")
ax1.legend()
ax2.step(numpy.linspace(-4,4,100),find_weight_z(numpy.linspace(-4,4,100)),where='mid',color='black')
ax2.set_xlabel("$Z$")
ax2.set_ylabel("weight($Z$)")
plt.savefig("weighting_z.pdf")

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))

var = {
    0:"$X$"
    ,1:"$Y$"
    ,2:"$Z$"
}

bins = numpy.linspace(-4,4,50)
ax1.hist(xH0[:,0].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax1.hist(xH1[:,0].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax1.hist(xH0[:,0].numpy(),histtype='stepfilled',alpha=0.5,weights=find_weight_z(xH0[:,2].numpy()),label="$H_{1}$ z weighted",color='green',bins=bins)

ax2.hist(xH0[:,1].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax2.hist(xH1[:,1].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax2.hist(xH0[:,1].numpy(),histtype='stepfilled',alpha=0.5,weights=find_weight_z(xH0[:,2].numpy()),label="$H_{1}$ z weighted",color='green',bins=bins)

ax3.hist(xH0[:,2].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax3.hist(xH1[:,2].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax3.hist(xH0[:,2].numpy(),histtype='stepfilled',alpha=0.5,weights=find_weight_z(xH0[:,2].numpy()),label="$H_{0}$ z weighted",color='green',bins=bins)

#plt.hist(xH0[:,2].numpy(),bins=bins_z,histtype='stepfilled', alpha = 0.5,weights = find_weight_z(xH0[:,2].numpy()),label="$H_{0}$ weighted",color='green')
#plt.xlabel("$Z$")


ax1.set_xlabel(var[0])
ax2.set_xlabel(var[1])
ax3.set_xlabel(var[2])

plt.tight_layout()
plt.legend()
plt.savefig("compare_1.pdf")

In [ ]:

kin_weights = find_weight_z(xH0[:,2].numpy())
kin_weights=numpy.append(kin_weights,[1. for n in range(n_events)])
print(kin_weights)
kin_weights = torch.reshape(torch.from_numpy(numpy.array(kin_weights)).type(torch.double),(2*n_events,1))

print(kin_weights.shape)
print(net_main(x).shape)
print(f.shape)
def custom_Loss(input, target):
  x, y = input, target
  
  log = lambda x: torch.log(x*(1-1e-8) + 1e-8)
  #return torch.mean(-kin_weights * (y*log(x) + (1-y)*log(1-x)))
  return torch.sum(-kin_weights * (y*log(x) + (1-y)*log(1-x)))/torch.sum(kin_weights)


def custom_Loss_2(input,target)
  x, y = input, target
  log = lambda x: torch.log(x*(1-1e-8) + 1e-8)
  
    

print(custom_Loss(net_main(x),f).detach().numpy().tolist())

In [ ]:
MAXSTEPS=500
step = 0

losses = []
losses_1 = []
losses_2 = []
lamb=5000
#while MSE>0.1: #for t in range(n_epochs):

while step < MAXSTEPS:

    step+=1

    prediction_main  = net_main(x)     # input x and predict based on x
    #weights = weight(x)
    
    #loss1 =  loss_func(prediction_main, f)
    #loss2 =  loss_func(xH0,xH0)
    #if step%100==0: loss2 = custom_Loss(xH0,xH0) # want to keep the same as initial distribution!
    #print(loss2.statistic)
    loss  =  custom_Loss(prediction_main,f)  
    
    optimizer_main.zero_grad()   # clear gradients for next train
    #optimizer_other.zero_grad() 
    loss.backward()         # backpropagation, compute gradients, 
                            # this doesn't work for us because we include 
                            # a term that depends on the parameters but is not connected 
                            # to things that are known about :/ 
    optimizer_main.step()        # apply gradients
    #optimizer_other.step() 
    
    losses.append(loss.detach().numpy().tolist())
    
    if step%100==0: print("Just finished iteration ", step)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))

ax1.plot(range(0,len(losses)),losses)#label = "$L_{1}+\lambda L_{2}$")
#ax2.plot(range(0,len(losses)),losses_1,label = "$L_{1}$")
#ax3.plot(range(0,len(losses)),losses_2,label = "$L_{2}$")
plt.savefig("train.pdf")
#ax1.legend()
#ax2.legend()
#ax3.legend()

In [ ]:
# create a network which is just a sigmoid function
#net_wS = torch.nn.Sequential(
#    torch.nn.Sigmoid()
#)




In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))

xH0_n = xH0
xH1_n = xH1

weights = weight(xH0_n).numpy().flatten() #*weight_1(xH0_n[:,2:])[:,0].numpy()

#print(weights)


#print(xH0[:,0],xH0_n[:,0].shape)

index = 2

var = {
    0:"$X$"
    ,1:"$Y$"
    ,2:"$Z$"
}


weights_compund = weights*find_weight_z(xH0[:,2].numpy()) 

print(weights.shape)
print((find_weight_z(xH0[:,2].numpy())).shape)

bins = numpy.linspace(-4,4,50)

ax1.hist(xH0_n[:,0].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax1.hist(xH0[:,0].numpy(),histtype='stepfilled',alpha=0.5,weights=weights_compund,label="$H_{0}$ weighted (+z weight)",color='green',bins=bins)
ax1.hist(xH1_n[:,0].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax1.hist(xH0_n[:,0].numpy(),histtype='step',weights=weights,label="$H_{1}$ weighted",color='purple',bins=bins)

ax2.hist(xH0_n[:,1].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax2.hist(xH0[:,1].numpy(),histtype='stepfilled',alpha=0.5,weights=weights_compund,label="$H_{0}$ weighted (+z weight)",color='green',bins=bins)
ax2.hist(xH1_n[:,1].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax2.hist(xH0_n[:,1].numpy(),histtype='step',weights=weights,label="$H_{1}$ weighted",color='purple',bins=bins)

ax3.hist(xH0_n[:,2].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
ax3.hist(xH0[:,2].numpy(),histtype='stepfilled',alpha=0.5,weights=weights_compund,label="$H_{0}$ weighted (+z weight)",color='green',bins=bins)
ax3.hist(xH1_n[:,2].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
ax3.hist(xH0_n[:,2].numpy(),histtype='step',weights=weights,label="$H_{0}$ weighted",color='purple',bins=bins)

ax1.set_xlabel(var[0])
ax2.set_xlabel(var[1])
ax3.set_xlabel(var[2])


ax1.legend(loc='upper left',ncol=2)
plt.tight_layout()
plt.savefig("compare_2.pdf")

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(18,12))


bins = numpy.linspace(-4,4,50)


axs[0][0].hist(xH0_n[xH0_n[:,2]>1,0].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
axs[0][0].hist(xH1_n[xH1_n[:,2]>1,0].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)

axs[0][0].hist(xH0_n[xH0_n[:,2]>1,0].numpy(),histtype='step',weights=weights[xH0_n[:,2]>1],label="$H_{1}$ weighted",color='green',bins=bins)

#ax2.hist(xH0_n[:,1].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
#ax2.hist(xH1_n[:,1].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
#ax2.hist(xH0_n[:,1].numpy(),histtype='step',weights=weights,label="$H_{1}$ weighted",color='green',bins=bins)

#ax3.hist(xH0_n[:,2].numpy(),histtype='step',label="$H_{0}$",color='red',bins=bins)
#ax3.hist(xH1_n[:,2].numpy(),histtype='step',label="$H_{1}$",color='blue',bins=bins)
#ax3.hist(xH0_n[:,2].numpy(),histtype='step',weights=weights,label="$H_{1}$ weighted",color='green',bins=bins)

axs[0][0].set_xlabel(var[0])
#ax2.set_xlabel(var[1])
#ax3.set_xlabel(var[2])

plt.legend()
plt.tight_layout()

In [ ]:
           
#pH0 = multivariate_normal([mu_H0_x,mu_H0_y,mu_H0_z], makeCovariance(sigma_H0_x,sigma_H0_y,sigma_H0_z))
#pH1 = multivariate_normal([mu_H1_x,mu_H1_y,mu_H1_z], makeCovariance(sigma_H1_x,sigma_H1_y,sigma_H1_z), allow_singular=True)


pH0_new_1 = multivariate_normal([mu_H0_x,mu_H0_y,mu_H0_z+1], makeCovariance(sigma_H0_x,sigma_H0_y,sigma_H0_z))
pH0_new_2 = multivariate_normal([mu_H0_x,mu_H0_y,mu_H0_z+1], makeCovariance(sigma_H0_x,sigma_H0_y,1.2))
pH0_new_3 = multivariate_normal([mu_H0_x,mu_H0_y,mu_H0_z-2], makeCovariance(sigma_H0_x,sigma_H0_y,0.1))


xH01 = pH0_new_1.rvs(size=n_events); xH01 = torch.reshape(torch.from_numpy(numpy.array(xH01)).type(torch.double),(n_events,3))
xH02 = pH0_new_2.rvs(size=n_events); xH02 = torch.reshape(torch.from_numpy(numpy.array(xH02)).type(torch.double),(n_events,3))
xH03 = pH0_new_3.rvs(size=n_events); xH03 = torch.reshape(torch.from_numpy(numpy.array(xH03)).type(torch.double),(n_events,3))

ig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(18,6))


weights1 = weight(xH01).numpy().flatten() #*weight_1(xH0_n[:,2:])[:,0].numpy()
weights2 = weight(xH02).numpy().flatten()
weights3 = weight(xH03).numpy().flatten()

ax1.hist(xH01[:,0].numpy(),histtype='step',label="$H_{0} 1$",color='red',bins=bins)
ax1.hist(xH01[:,0].numpy(),histtype='step',label="$H_{0} 1 weighted$",color='red',bins=bins,weights=weights1,linestyle='--')
ax1.hist(xH02[:,0].numpy(),histtype='step',label="$H_{0} 2 $",color='blue',bins=bins)
ax1.hist(xH02[:,0].numpy(),histtype='step',label="$H_{0} 2 weighted$",color='blue',bins=bins,weights=weights2,linestyle='--')
ax1.hist(xH03[:,0].numpy(),histtype='step',label="$H_{0} 3 $",color='green',bins=bins)
ax1.hist(xH03[:,0].numpy(),histtype='step',label="$H_{0} 3 weighted$",color='green',bins=bins,weights=weights3,linestyle='--')

ax2.hist(xH01[:,1].numpy(),histtype='step',label="$H_{0} 1$",color='red',bins=bins)
ax2.hist(xH01[:,1].numpy(),histtype='step',label="$H_{0} 1 weighted$",color='red',bins=bins,weights=weights1,linestyle='--')
ax2.hist(xH02[:,1].numpy(),histtype='step',label="$H_{0} 2 $",color='blue',bins=bins)
ax2.hist(xH02[:,1].numpy(),histtype='step',label="$H_{0} 2 weighted$",color='blue',bins=bins,weights=weights2,linestyle='--')
ax2.hist(xH03[:,1].numpy(),histtype='step',label="$H_{0} 3 $",color='green',bins=bins)
ax2.hist(xH03[:,1].numpy(),histtype='step',label="$H_{0} 3 weighted$",color='green',bins=bins,weights=weights3,linestyle='--')

ax3.hist(xH01[:,2].numpy(),histtype='step',label="$H_{0} 1$",color='red',bins=bins)
ax3.hist(xH01[:,2].numpy(),histtype='step',label="$H_{0} 1 weighted$",color='red',bins=bins,weights=weights1,linestyle='--')
ax3.hist(xH02[:,2].numpy(),histtype='step',label="$H_{0} 2 $",color='blue',bins=bins)
ax3.hist(xH02[:,2].numpy(),histtype='step',label="$H_{0} 2 weighted$",color='blue',bins=bins,weights=weights2,linestyle='--')
ax3.hist(xH03[:,2].numpy(),histtype='step',label="$H_{0} 3 $",color='green',bins=bins)
ax3.hist(xH03[:,2].numpy(),histtype='step',label="$H_{0} 3 weighted$",color='green',bins=bins,weights=weights3,linestyle='--')

ax1.set_xlabel(var[0])
ax2.set_xlabel(var[1])
ax3.set_xlabel(var[2])


ax1.legend(loc='upper left',ncol=2)
plt.tight_layout()
plt.savefig("compare_fake_hgg.pdf")